In [ ]:
from google.cloud import bigquery

client = bigquery.Client()

encounters="""
SELECT
    class.code as encounter_class,
    period.start as start_timestamp,
    period.end as end_timestamp, 
    TIMESTAMP_DIFF(period.end, period.start, HOUR) as length_of_stay_in_hours
FROM 
    hc_dataset.Encounter
WHERE
    period.end >= period.start
ORDER BY
    4 DESC
LIMIT 10
"""
df = client.query(encounters).to_dataframe()
df.head()

In [ ]:
query2="""
SELECT
  cc.code loinc_code,
  cc.display loinc_name,
  approx_quantiles(round(o.valueQuantity.value,1),4) as quantiles,
  count(*) as num_obs
FROM
  hc_dataset.Observation o, o.code.coding cc
WHERE
  cc.system like '%loinc%' and lower(cc.display) like '%cholesterol%'
GROUP BY 1,2
ORDER BY 4 desc
"""
df2 = client.query(query2).to_dataframe()
df2.head()

In [ ]:
query3="""
SELECT
  encounter_class,
  APPROX_QUANTILES(num_encounters, 4) num_encounters_quantiles
FROM (
  SELECT
    class.code encounter_class,
    subject.reference patient_id,
    COUNT(DISTINCT id) AS num_encounters
  FROM
    hc_dataset.Encounter
  GROUP BY
    1,2
  )
GROUP BY 1
ORDER BY 1
"""
df3 = client.query(query3).to_dataframe()
df3.head()

In [ ]:
query4="""
SELECT
  class.code encounter_class,
  ROUND(AVG(TIMESTAMP_DIFF(period.end, period.start, MINUTE)),1) as avg_minutes
FROM
  hc_dataset.Encounter
WHERE
  period.end >= period.start
GROUP BY
  1
ORDER BY
  2 DESC
  """
df4 = client.query(query4).to_dataframe()
df4.head()

In [ ]:
df4.plot(kind='bar', x='encounter_class', y='avg_minutes');